<a href="https://colab.research.google.com/github/mpky/tb_colab/blob/master/ConvNet_sentdex_tutorial_20190417.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# download and unzip ngrok
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2019-04-17 11:48:48--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.45.111.123, 52.204.136.9, 34.231.75.48, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.45.111.123|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14977695 (14M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  14.28M  82.6MB/s    in 0.2s    

2019-04-17 11:48:49 (82.6 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [14977695/14977695]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [0]:
# run tensorboard
LOG_DIR = './log'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

In [0]:
# run ngrok to tunnel tensorboard port 6006
get_ipython().system_raw('./ngrok http 6006 &')

In [4]:
# get the url for TB page
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://bbd975a0.ngrok.io


In [5]:
# mount my drive
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
import pickle
import time

In [7]:
!pwd

/content


In [0]:
X = pickle.load(open("/content/gdrive/My Drive/Colab_Notebooks/Colab_Projects/sentdex_tutorial/X.pickle", "rb"))
y = pickle.load(open("/content/gdrive/My Drive/Colab_Notebooks/Colab_Projects/sentdex_tutorial/y.pickle", "rb"))

In [9]:
NAME = "Cats-vs-dog-cnn-64x2-50epochs-{}".format(int(time.time()))

tensorboard = TensorBoard(log_dir='./log/{}'.format(NAME))

# need to scale
X = X / 255

model = Sequential()
model.add(Conv2D(64, (3,3), input_shape=X.shape[1:]))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation("relu"))

model.add(Dense(1))
model.add(Activation("sigmoid"))

model.compile(loss='binary_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

model.fit(X,y,batch_size=32,validation_split=0.3, epochs=50, callbacks=[tensorboard])

Instructions for updating:
Colocations handled automatically by placer.
Train on 17462 samples, validate on 7484 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/50
17462/17462 [==============================] - 8s 436us/sample - loss: 0.6257 - acc: 0.6425 - val_loss: 0.5728 - val_acc: 0.7068
Epoch 2/50
17462/17462 [==============================] - 4s 221us/sample - loss: 0.5227 - acc: 0.7414 - val_loss: 0.5200 - val_acc: 0.7461
Epoch 3/50
17462/17462 [==============================] - 4s 232us/sample - loss: 0.4711 - acc: 0.7755 - val_loss: 0.5014 - val_acc: 0.7557
Epoch 4/50
17462/17462 [==============================] - 4s 216us/sample - loss: 0.4363 - acc: 0.7961 - val_loss: 0.5092 - val_acc: 0.7560
Epoch 5/50
17462/17462 [==============================] - 4s 210us/sample - loss: 0.4060 - acc: 0.8106 - val_loss: 0.4980 - val_acc: 0.7667
Epoch 6/50
17462/17462 [==============================] - 4s 211us/sample - loss: 0.3680 - acc: 0.8345 - val_loss: 0.4880 - val_acc

In [10]:
!ps

    PID TTY          TIME CMD
      1 ?        00:00:00 run.sh
      8 ?        00:00:01 node
     23 ?        00:00:03 node
     33 ?        00:00:07 jupyter-noteboo
    124 ?        00:00:00 tail
    139 ?        00:03:51 python3
    163 ?        00:00:02 tensorboard
    170 ?        00:00:01 ngrok
    209 ?        00:00:00 drive
    275 ?        00:00:00 drive
    323 ?        00:00:00 tail
    324 ?        00:00:00 grep
    440 ?        00:00:00 ps


In [0]:
!kill 163 170

In [12]:
!ps

    PID TTY          TIME CMD
      1 ?        00:00:00 run.sh
      8 ?        00:00:01 node
     23 ?        00:00:03 node
     33 ?        00:00:07 jupyter-noteboo
    124 ?        00:00:00 tail
    139 ?        00:03:52 python3
    209 ?        00:00:00 drive
    275 ?        00:00:00 drive
    323 ?        00:00:00 tail
    324 ?        00:00:00 grep
    458 ?        00:00:00 ps
